# Reduce ImageCollection

In [ ]:
import ee, geetools
from geetools.utils import initialize_documentation

initialize_documentation()

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geetools/blob/main/docs/usage/reduce.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/gee-community/geetools/blob/main/docs/usage/reduce.ipynb)

THe Earth Engine API provides 2 ways to reduce images: `reduceRegion` and `reduceRegions`. `geetools` is making these methods also available for `ee.ImageCollection` objects.

## Set up environment

Install all the required libs if necessary and perform the import statements upstream.

In [ ]:
# uncomment if installation of libs is necessary
# !pip install earthengine-api geetools

In [ ]:
import ee
import geetools #noqa: F401
import geopandas as gpd
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
# uncomment if initialization is required
# ee.Initialize()

## Example data 

The following examples rely on a `ee.FeatureCollection` composed of three ecoregion features that define regions by which to reduce image data. The ImageCollection data loads the modis vegetation indicies and subset the 2010 2020 decade of images.

In [ ]:
## Import the example feature collection and drop the data property.
ecoregions = (
    ee.FeatureCollection("projects/google/charts_feature_example")
    .select(["label", "value", "warm"])
)


## Load MODIS vegetation indices data and subset of 4 images.
vegIndices = (
    ee.ImageCollection("MODIS/061/MOD13A1")
    .filter(ee.Filter.date("2010-01-01", "2010-02-28"))
    .select(["NDVI", "EVI"])
)

## Reduce over single region

Using `reduceRegion` you can reduce an `ee.ImageCollection` over a single region. 
The function will return a `ee.Dictionary` with the reduced values of each band grouped under each image Id as key.

It will return a `ee.Dictionary` with the following shape: 

```
{
    "image1": {"band1": value1, "band2": value2, ...},
    "image2": {"band1": value1, "band2": value2, ...},
}
```

where `image*`is the id of the image as per specified property (casted to string) and `band*` is the name of the band.

In [ ]:
result = vegIndices.geetools.reduceRegion(
    reducer = ee.Reducer.mean(),
    idProperty = "system:time_start",
    idType = ee.Date,
    geometry = ecoregions.filter(ee.Filter.eq("label", "Forest")).geometry(),
    scale = 500
)
result.getInfo()

Then a user can easily transform this data into a dataframe and use any tools from the Python ecosystem:

In [ ]:
df = pd.DataFrame(result.getInfo()).transpose()
df.head(15)

## Reduce over muliple regions

Using `reduceRegions` you can reduce an `ee.ImageCollection` over multiple regions.
The result will be shaped as a `ee.FeatureCollection` with 2 primary keys.
- The `idProperty` as key for images stored in final feature as `image_id`
- The id of the feature stored in the final features as `feature_id`.

Each feature will have the same properties as the original feature collection + the reduced value of the corresponding image over the feature geometry.
The user can specify all the parameter of the reduction and specify which image property will be used as the id of the image.

In [ ]:
result = vegIndices.geetools.reduceRegions(
    reducer = ee.Reducer.mean(),
    idProperty = "system:time_start",
    idType = ee.Date,
    collection = ecoregions,
    scale = 500
)

# we can display the result as a table using geopandas
gdf = gpd.GeoDataFrame.from_features(result.getInfo()["features"])
gdf.head(15)

From this you can easily create chrono mapping of the regions or more custom figures that are not covered by the `plot_*` methods:

In [ ]:
# Create a figure with 2 rows and 3 columns
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 6))  # Adjust figsize as needed

# Flatten the 2D array of axes for easier access, if needed
axes_flat = axes.flatten()

# get a list of all the available dates
dates = vegIndices.aggregate_array("system:time_start").distinct()

# Plot the data
for i in range(3):
    ax = axes_flat[i]
    image_id = ee.Date(dates.get(i)).format("YYYY-MM-dd'T'HH-mm-ss")
    fc = result.filter(ee.Filter.eq("image_id", image_id))
    fc.geetools.plot(ax=ax, cmap="viridis", property="NDVI")
    ax.set_title(image_id.getInfo())